In [11]:
import numpy as np
import pandas as pd

In [12]:
filepath = '../results/openrarity_scores'

In [13]:
def get_df(name: str) -> pd.DataFrame:
    df = pd.read_csv(filepath + '/' + name + '_openrarity_scores.csv')
    return df

In [14]:
df = get_df('KILLABEARS')

In [15]:
factor = 1437.41 / 0.502271
factor

2861.8216062643473

In [16]:
real_rarity = {
    1819: 1435.30,
    7711: 1446.19,
    8394: 1447.80,
    7715: 1472.71,
    668:  34773.85,
}

In [17]:
df[df['Token Id'] == 668]

,Unnamed: 0,Mouth information value,Eyebrows information value,Eyelids information value,Eyes information value,Fur information value,Background information value,Clothing information value,Hair information value,Eyewear information value,Head Gear information value,Mask information value,Special Head information value,Trait count information value,Unique Trait count,Token Id,Rarity score
667,667,2.967896,1.44994,1.244199,2.362756,2.251394,3.259662,4.783742,0.137504,0.36945,0.451715,5.795715,0.062361,4.016105,0,668,0.479037


In [18]:
sum([10.70275, 10.70275,10.70275,10.70275,10.70275,10.70275,13.287712,10.70275])

88.20696199999999

In [19]:
from os import listdir
from os.path import isfile, join
from collections import ChainMap
import pandas as pd
import numpy as np
from natsort import natsorted
from tqdm import tqdm
import json


def metadataJsonToDF(metadata):
    traits = []
    problems = {'No Traits tokens': [], 'Wrong Traits format tokens': []}
    for token in metadata:
        if token['traits']:
            token['traits'].append({'value': token['collectionTokenId'], 'trait_type': 'Token Id'})
            trait_list = []
            for d in token['traits']:
                try:
                    trait_list.append({d['trait_type']: d['value']})
                except:
                    problems['Wrong Traits format tokens'].append(token['collectionTokenId'])
                    trait_list.append({'Noname Trait': d['value']})
            traits.append([dict(ChainMap(*trait_list))])
        else:
            problems['No Traits tokens'].append(token['collectionTokenId'])
            token['traits'].append({'value': token['collectionTokenId'], 'trait_type': 'Token Id'})
            traits.append([dict(ChainMap(*[{d['trait_type']: d['value']} for d in token['traits']]))])
    traitsDF = pd.concat([pd.DataFrame(trait) for trait in traits]).reset_index(drop=True)
    return traitsDF, problems


def rarityToolsScore(traits_data, weighted=False):
    tokenIdColumn = traits_data['Token Id']
    traits_data = traits_data.drop('Token Id', axis=1)
    if not('Trait count' in traits_data.columns):
        traits_data['Trait count'] = traits_data.count(axis=1)
    traits_data = traits_data.replace(np.nan, "None")
    rarity_values = {}
    for attribute in traits_data.columns:
        value_counts = traits_data[attribute].value_counts()
        total_token_count = len(traits_data)
        if weighted is False:
            rarity = 1 / (value_counts / total_token_count)
        elif weighted is True:
            rarity = total_token_count / (value_counts * len(value_counts))
        rarity_values[attribute + ' rarity score'] = traits_data[attribute].map(rarity)

    rarityScores = pd.DataFrame(rarity_values)
    rarityScores['Rarity score'] = rarityScores.sum(axis="columns")
    rarityScores['Token Id'] = tokenIdColumn
    return rarityScores

# metadataPath = '../data/metadata_quicknode'
metadataPath = '/Users/maksimshuklin/Desktop/metadata'
rarityToolsScoresPath =  '../results/raritytools_scores'
onlyfiles = natsorted([f for f in listdir(metadataPath) if isfile(join(metadataPath, f))])

In [22]:
fileName = 'BAYC_metadata.json'
problems = {}

print(f"{fileName.split('_')[0]} - start processing.")

with open(metadataPath + '/' + fileName, 'r') as file:
    metadataDF, collection_problems = metadataJsonToDF(json.load(file))
    problems[fileName[:fileName.find("_")]] = collection_problems
scores = rarityToolsScore(metadataDF, weighted=False) # added weighed=True 
scores.to_csv(rarityToolsScoresPath + '/' + fileName[:fileName.find("_")] + '_raritytools_scores.csv')

BAYC - start processing.


In [23]:
metadataDF

,Token Id,Eyes,Mouth,Clothes,Fur,Background,Earring,Hat
0,0,X Eyes,Discomfort,Striped Tee,Robot,Orange,Silver Hoop,NaN
1,1,Blue Beams,Grin,Vietnam Jacket,Robot,Orange,NaN,NaN
2,2,3d,Bored Cigarette,NaN,Robot,Aquamarine,NaN,Sea Captain's Hat
3,3,Bored,Tongue Out,Bone Necklace,Cheetah,Purple,NaN,NaN
4,4,Closed,Phoneme L,Navy Striped Tee,Golden Brown,Blue,NaN,Party Hat 2
...,...,...,...,...,...,...,...,...
9995,9995,Closed,Bored,Smoking Jacket,Pink,Gray,NaN,NaN
9996,9996,3d,Dumbfounded,Guayabera,Dark Brown,New Punk Blue,Silver Hoop,NaN
9997,9997,Bored,Grin Multicolored,Sailor Shirt,Black,Purple,NaN,Halo
9998,9998,Heart,Bored Unshaven Cigarette,Bayc T Red,Brown,Yellow,NaN,NaN


In [43]:
fileName = 'VFT_metadata.json'

with open(metadataPath + '/ + fileName, 'r') as file:
    metadataDF, _ = metadataJsonToDF(json.load(file))

In [44]:
metadataDF['Access Remaining'].value_counts()

2 of 3     73
9 of 15    73
1 of 2     52
5 of 15    32
1 of 3     25
8 of 15    19
0 of 2     13
0 of 3      2
6 of 15     1
Name: Access Remaining, dtype: int64

In [33]:
metadataDF[metadataDF['Token Id'] == '0']['Access Remaining']
scores[scores['Token Id'] == '6874']['Access Remaining rarity score']

6874    197.211538
Name: Access Remaining rarity score, dtype: float64

In [40]:
metadataDF[metadataDF['Access Remaining'] == '1 of 2'].shape

(52, 12)

In [41]:
metadataDF['Access Remaining'].value_counts()

2 of 3     73
9 of 15    73
1 of 2     52
5 of 15    32
1 of 3     25
8 of 15    19
0 of 2     13
0 of 3      2
6 of 15     1
Name: Access Remaining, dtype: int64

In [32]:
metadataDF[metadataDF['Token Id'] == '6874']

,Token Id,Character,Edition #,Token Benefit,Spectacular Type,Rarity,Gary Originally Owned,Access Remaining,Access Type,Just Art,Gifts Remaining,Redemptions
6874,6874,Part Of The Team Dream,1 of 1,Access,NaN,NaN,NaN,1 of 2,One on One,Just Art,NaN,NaN


In [215]:
from os import listdir
from os.path import isfile, join
from collections import ChainMap
import pandas as pd
import numpy as np
from natsort import natsorted
from tqdm import tqdm
import json


def metadataJsonToDF(metadata):
    traits = []
    problems = {'No Traits tokens': [], 'Wrong Traits format tokens': []}
    for token in metadata:
        if token['traits']:
            token['traits'].append({'value': token['collectionTokenId'], 'trait_type': 'Token Id'})
            trait_list = []
            for d in token['traits']:
                try:
                    trait_list.append({d['trait_type']: d['value']})
                except:
                    problems['Wrong Traits format tokens'].append(token['collectionTokenId'])
                    trait_list.append({'Noname Trait': d['value']})
            traits.append([dict(ChainMap(*trait_list))])
        else:
            problems['No Traits tokens'].append(token['collectionTokenId'])
            token['traits'].append({'value': token['collectionTokenId'], 'trait_type': 'Token Id'})
            traits.append([dict(ChainMap(*[{d['trait_type']: d['value']} for d in token['traits']]))])
    traitsDF = pd.concat([pd.DataFrame(trait) for trait in traits]).reset_index(drop=True)
    return traitsDF, problems


def countUniqueTraits(traits_data):
    uniqueTraitCount = [0 for _ in range(len(traits_data))]
    for attribute in traits_data.columns[:-1]:
        duplicates = traits_data[attribute].duplicated(keep=False)
        duplicates = duplicates[duplicates == False]
        if not duplicates.empty:
            unique_values_ids = duplicates.index
            for id in unique_values_ids:
                uniqueTraitCount[id] += 1
    return uniqueTraitCount


def openRarityScore(traits_data):
    tokenIdColumn = traits_data['Token Id']
    traits_data = traits_data.drop('Token Id', axis=1)
    if not('Trait count' in traits_data.columns):
        traits_data['Trait count'] = traits_data.count(axis=1)
    traits_data = traits_data.replace(np.nan, "None")

    uniqueTraitCount = countUniqueTraits(traits_data)

    informationValues = {}
    #expectedValues = {}
    total_token_count = len(traits_data)
    total_attribute_count = len(traits_data.columns)

    for attribute in traits_data.columns:
        value_counts = traits_data[attribute].value_counts()
        prob = value_counts / total_token_count
        logProb = (1 / prob).apply(np.log2)
        expectInform = logProb
        #expectNorm = prob * logProb
        informationValues[attribute + ' information value'] = traits_data[attribute].map(expectInform)
        #expectedValues[attribute + ' expected value'] = traits_data[attribute].map(expectNorm)

    rarityScores = pd.DataFrame(informationValues)
    #normScores = pd.DataFrame(expectedValues)
    #rarityScores['Rarity score'] = rarityScores.sum(axis="columns") / normScores.sum().sum()

    rarityScores['Information content'] = rarityScores.sum(axis="columns")
    rarityScores['Norm factor'] = rarityScores.sum(axis="columns").sum(axis="rows") / total_token_count
    rarityScores['Unique Trait count'] = uniqueTraitCount 
    rarityScores['Token Id'] = tokenIdColumn
    rarityScores['Rarity score'] = (rarityScores['Information content'] + rarityScores['Unique Trait count'] * (total_attribute_count - 1) * np.log2(total_token_count)) / rarityScores['Norm factor']
    rarityScores = rarityScores.drop(['Information content', 'Norm factor'], axis=1)
    return rarityScores


metadataPath = '../data/metadata'
openRarityScoresPath = '../results/openrarity_scores'
onlyfiles = natsorted([f for f in listdir(metadataPath) if isfile(join(metadataPath, f))])

In [222]:
problems = {}
fileName = 'MICE' + '_metadata.json'
print(f"{fileName.split('_')[0]} - start processing.")
with open(metadataPath + '/' + fileName, 'r') as file:
    metadataDF, collection_problems = metadataJsonToDF(json.load(file))
    problems[fileName[:fileName.find("_")]] = collection_problems
scores = openRarityScore(metadataDF)

MICE - start processing.


In [223]:
scores

,character information value,nose information value,mouth information value,eyes information value,earrings information value,neck information value,whiskers information value,hat information value,burned information value,Trait count information value,Unique Trait count,Token Id,Rarity score
0,3.296191,2.311148,2.825210,2.375572,0.151042,3.140507,1.742265,3.923578,1.494516,0.0,0,0,0.578906
1,3.149441,2.311148,2.769059,2.375572,0.151042,0.613520,1.742265,0.932086,1.494516,0.0,0,1,0.423114
2,3.089267,2.311148,2.774972,3.660178,0.151042,0.613520,1.095111,3.289122,1.494516,0.0,0,2,0.503176
3,3.296191,2.236503,2.786870,4.248793,0.151042,3.167475,3.159718,0.932086,1.494516,0.0,0,3,0.584710
4,3.089267,2.345931,2.786870,2.469130,0.151042,0.613520,1.095111,0.932086,1.494516,0.0,0,4,0.407833
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,3.149441,2.345931,2.825210,2.375572,0.151042,0.613520,1.095111,0.932086,1.494516,0.0,0,9995,0.407968
9996,3.149441,2.236503,2.825210,2.375572,0.151042,0.613520,1.095111,3.923578,1.494516,0.0,0,9996,0.486446
9997,2.810966,2.359194,2.827256,4.483581,0.151042,0.613520,1.742265,0.932086,1.494516,0.0,0,9997,0.474191
9998,3.089267,2.345931,2.774972,2.469130,4.733124,0.613520,1.095111,4.187050,1.494516,0.0,0,9998,0.620910


In [224]:
def get_score(token_id: int):
    return scores[scores['Token Id'] == str(token_id)]['Rarity score'].item()

In [88]:
for i, row in metadataDF.iterrows():
    print(row.to_dict())
    if i == 0:
        break

{'Token Id': '0', 'Character': '"Content" Condor', 'Edition #': '1 of 1', 'Token Benefit': 'Admission', 'Spectacular Type': 'Bubble Gum', 'Rarity': 'Spectacular', 'Gary Originally Owned': nan, 'Access Remaining': nan, 'Access Type': nan, 'Just Art': nan, 'Gifts Remaining': nan, 'Redemptions': nan}


In [87]:
row = row.to_dict()
row = set(map(lambda x: row[x], row))
row

{'"Content" Condor',
 '0',
 '1 of 1',
 'Admission',
 'Bubble Gum',
 'Spectacular',
 nan}

In [92]:
row1 = set({1, 2, 3})
row2 = set({1, 2, 4})
(row1 | row2)

{1, 2, 3, 4}

In [95]:
a = np.zeros(5)
a[0] = 1
a

array([1., 0., 0., 0., 0.])

In [99]:
pd.DataFrame({
    'a': np.array([1.0, 2.4, 3]),
    'b': [2, 3, 4]
})

,a,b
0,1.0,2
1,2.4,3
2,3.0,4


In [4]:
NFTGO_API_KEY = '54804bbd-44b7-4f50-8229-3c5fa3db2dac'

In [46]:
URL = 'https://data-api.nftgo.io/eth/v1/nft/{contractAddress}/{tokenId}/info'

In [49]:
request_url = URL.format(
    contractAddress=0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,
    tokenId=4495
)

In [52]:
request_url

'https://data-api.nftgo.io/eth/v1/nft/1074999140385142990242729887528841825919887143229/4495/info'

In [51]:
curl 'https://data-api.nftgo.io/<your-request-url> -H X-API-KEY:<YOUR-API-KEY>

curl: (6) Could not resolve host: request_url


In [57]:
!curl 'https://data-api.nftgo.io/eth/v1/nft/1074999140385142990242729887528841825919887143229/4495/info' -H X-API-KEY:'54804bbd-44b7-4f50-8229-3c5fa3db2dac'

{"detail":[{"loc":["path","contract_address"],"msg":"ensure this value has at most 42 characters","type":"value_error.any_str.max_length","ctx":{"limit_value":42}}]}

In [1]:
import requests

In [71]:
requests.get(
    request_url,
    headers={
        'X-API-KEY': '54804bbd-44b7-4f50-8229-3c5fa3db2dac'
    }
).content

b'{"detail":[{"loc":["path","contract_address"],"msg":"ensure this value has at most 42 characters","type":"value_error.any_str.max_length","ctx":{"limit_value":42}}]}'

In [6]:
print(NFTGO_API_KEY)

54804bbd-44b7-4f50-8229-3c5fa3db2dac


In [78]:
tokenId = 2
contractAddress = '0xb452ff31b35dee74f2fdfd5194b91af1bad07b91'
url = f'https://data-api.nftgo.io/eth/v1/nft/{contractAddress}/{tokenId}/info'

In [79]:
response = requests.get(
    url=url,
    headers={
        'X-API-KEY': NFTGO_API_KEY
    },
)

In [80]:
response.content

b'{"blockchain":"ETH","collection_name":"XBorg NFT","collection_slug":"x-borg-nft","collection_opensea_slug":"xborg-nft","collection_logo":null,"contract_type":"ERC721","contract_address":"0xb452ff31b35dee74f2fdfd5194b91af1bad07b91","token_id":"2","name":"XBorg #2","description":"XBorgs\xe2\x80\x99 are a genesis collection of 2,500 NFTs on Ethereum blockchain and a brand mascot of VISPX. The XBorg NFTs are made up of hundreds of hand-drawn exciting visual traits that make them unique. Known for its powerhouse of features, holding this unique NFT unlocks 10+ utilities including","image":"https://lh3.googleusercontent.com/XOtHf9H4t5rqO1Hmw0oz5C4dYhO9i2eIVKTs_q8_f45uEA241n2K91fECvQa3ORqnnEUCYzkZJQr9zP2q_kpIBziy_BATZUEng","animation_url":null,"owner_addresses":["0x1ee4c5c57a442ef86605dcaa204f5c48843f1825"],"traits":[{"type":"Class","value":"Common","percentage":0.6},{"type":"Background","value":"Attitude","percentage":0.0204},{"type":"Skin","value":"Purple","percentage":0.0524},{"type":"An

In [92]:
!curl https://data-api.nftgo.io/eth/v1/nft/0xb452ff31b35dee74f2fdfd5194b91af1bad07b91/2/info -H X-API-KEY:54804bbd-44b7-4f50-8229-3c5fa3db2dac | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1751    0  1751    0     0   3329      0 --:--:-- --:--:-- --:--:--  3360
{
  "blockchain": "ETH",
  "collection_name": "XBorg NFT",
  "collection_slug": "x-borg-nft",
  "collection_opensea_slug": "xborg-nft",
  "collection_logo": null,
  "contract_type": "ERC721",
  "contract_address": "0xb452ff31b35dee74f2fdfd5194b91af1bad07b91",
  "token_id": "2",
  "name": "XBorg #2",
  "description": "XBorgs’ are a genesis collection of 2,500 NFTs on Ethereum blockchain and a brand mascot of VISPX. The XBorg NFTs are made up of hundreds of hand-drawn exciting visual traits that make them unique. Known for its powerhouse of features, holding this unique NFT unlocks 10+ utilities including",
  "image": "https://lh3.googleusercontent.com/XOtHf9H4t5rqO1Hmw0oz5C4dYhO9i2eIVKTs_q8_f45uEA241n2K91fECvQa3ORqnnEUCYzkZJQr9zP2q_kpIBziy_BATZUEng",
  "

In [74]:
import requests

response = requests.get(
    url='https://data-api.nftgo.io/eth/v1/nft/0xb452ff31b35dee74f2fdfd5194b91af1bad07b91/2/info',
    headers={
        'X-API-KEY': NFTGO_API_KEY,
    }
)

In [75]:
response.content

b'{"blockchain":"ETH","collection_name":"XBorg NFT","collection_slug":"x-borg-nft","collection_opensea_slug":"xborg-nft","collection_logo":null,"contract_type":"ERC721","contract_address":"0xb452ff31b35dee74f2fdfd5194b91af1bad07b91","token_id":"2","name":"XBorg #2","description":"XBorgs\xe2\x80\x99 are a genesis collection of 2,500 NFTs on Ethereum blockchain and a brand mascot of VISPX. The XBorg NFTs are made up of hundreds of hand-drawn exciting visual traits that make them unique. Known for its powerhouse of features, holding this unique NFT unlocks 10+ utilities including","image":"https://lh3.googleusercontent.com/XOtHf9H4t5rqO1Hmw0oz5C4dYhO9i2eIVKTs_q8_f45uEA241n2K91fECvQa3ORqnnEUCYzkZJQr9zP2q_kpIBziy_BATZUEng","animation_url":null,"owner_addresses":["0x1ee4c5c57a442ef86605dcaa204f5c48843f1825"],"traits":[{"type":"Class","value":"Common","percentage":0.6},{"type":"Background","value":"Attitude","percentage":0.0204},{"type":"Skin","value":"Purple","percentage":0.0524},{"type":"An

In [85]:
import asyncio
import aiohttp
import requests
import threading


async def async_fetch(session, url):
    """
    Asyncronously fetch (get-request) single url using provided session
    :param session: aiohttp session object
    :param url: target http url
    :return: fetched text
    """
    async with session.get(url) as response:
        html = await response.text()
        return html


async def async_requests(urls):
    """
    Concurrently fetch provided urls using aiohttp
    :param urls: list of http urls ot fetch
    :return: list of fetched texts
    """
    fetched_texts: list[str] = []
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in urls:
            task = asyncio.create_task(async_fetch(session, url))
            tasks.append(task)
        for task in tasks:
            html = await task
            fetched_texts.append(html)
    return fetched_texts

In [91]:
kek = async_requests('http://example.com')
kek

/var/folders/tv/9n19nz5j7cq_q4j3vm06dp9w0000gn/T/ipykernel_65005/2145108653.py:1: RuntimeWarning: coroutine 'async_requests' was never awaited
  kek = async_requests('http://example.com')


<coroutine object async_requests at 0x1218c8240>

In [104]:
import requests
import json

def get_token_score(tokenId: int):
    url = f'https://data-api.nftgo.io/eth/v1/nft/{contractAddress}/{tokenId}/info'
    response = requests.get(
        url=url,
        headers={
            'X-API-KEY': NFTGO_API_KEY,
        }
    )
    json_response = json.loads(response.content.decode())
    return json_response['rarity']['score']

In [136]:
import pandas as pd

df = pd.DataFrame({
    'tokenId': [i for i in range(100)],
    'score': [get_token_score(i) for i in range(100)]
})


KeyboardInterrupt



In [252]:
import numpy as np

In [253]:
dima_df = pd.read_csv('../results/nftgo_scores/XBORG_nftgo_scores.csv')
my_df = pd.read_csv('../results/nftgo_scores/XBORG_raritytools_scores.csv')

In [254]:
correct = np.array(df['score'].tolist()[:100])
dimas = np.array(list(map(lambda x: round(x, 2), ours_df['Rarity score'].tolist()[:100])))
my = np.array(list(map(lambda x: round(x, 2), ours_df['Rarity score'].tolist()[:100])))

In [255]:
print(correct[:15])

[71.4  65.59 11.2   4.48  5.8  11.54 25.56  4.44 13.05  4.18 10.19 68.38
 11.31 66.96 11.82]


In [256]:
print(dimas[:15])

[78.34 51.32  8.68  3.47  4.49  8.95 19.89  3.45 10.12  3.25  7.9  53.52
  8.78 52.39  9.17]


In [257]:
my[:15]

array([78.34, 51.32,  8.68,  3.47,  4.49,  8.95, 19.89,  3.45, 10.12,
        3.25,  7.9 , 53.52,  8.78, 52.39,  9.17])

In [259]:
my == dimas

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [260]:
np.argsort(correct) == np.argsort(dimas)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False, False, False, False, False, False, False,
        True])

In [261]:
np.argsort(correct)

array([17, 70, 56, 82, 85, 15, 91, 66,  9, 69,  7,  3, 78, 68, 50, 72, 97,
       19,  4, 99, 18, 46, 43, 47, 55, 20, 87, 35, 98, 76, 93, 71, 28, 42,
       86, 24, 73, 95, 61, 10, 23, 60, 90,  2, 12, 77, 45,  5, 14, 83, 32,
       39,  8, 75, 49, 58, 62, 79, 81, 84, 21, 53, 92, 16, 64, 36, 22, 48,
       51, 96, 89, 94, 88,  6, 44, 38, 65, 31, 33, 37, 25, 59, 34, 41, 30,
       52, 29, 63, 67, 80, 40, 27, 54,  1, 13, 74, 57, 11, 26,  0])

In [262]:
np.argsort(ours)

array([17, 70, 56, 82, 85, 15, 91, 66,  9, 69,  7,  3, 78, 68, 50, 72, 97,
       19,  4, 99, 18, 46, 43, 47, 55, 20, 87, 35, 98, 76, 93, 71, 28, 42,
       86, 24, 73, 95, 61, 10, 23, 60, 90,  2, 12, 77, 45,  5, 14, 83, 32,
       39,  8, 75, 49, 58, 62, 79, 81, 84, 21, 53, 92, 16, 64, 36, 22, 48,
       51, 96, 89, 94, 88,  6, 44, 38, 65, 31, 33, 37, 25, 59, 34, 41, 30,
       52, 29, 63, 67, 80, 27,  1, 13, 74, 57, 11, 26, 40, 54,  0])

In [263]:
a = [40, 27, 54,  1, 13, 74, 57, 11, 26,  0]
b = [27,  1, 13, 74, 57, 11, 26, 40, 54,  0]
dp = [[0 for i in range(10 + 1)] for j in range(10 + 1)]
for i in range(1, 10 + 1):
    for j in range(1, 10 + 1):
        if a[i - 1] == b[j - 1]:
            dp[i][j] = max(dp[i][j], dp[i - 1][j - 1]) + 1
        dp[i][j] = max(dp[i][j], dp[i - 1][j])
        dp[i][j] = max(dp[i][j], dp[i][j - 1])
dp[10][10]

8

In [264]:
for i in range(100):
    for j in range(100):
        if ours[i] > ours[j] and correct[i] < correct[j]:
            print(f'{i} -- {j}')
        if ours[i] < ours[j] and correct[i] > correct[j]:
            print(f'{i} -- {j}')

1 -- 40
1 -- 54
11 -- 40
11 -- 54
13 -- 40
13 -- 54
26 -- 40
26 -- 54
27 -- 40
40 -- 1
40 -- 11
40 -- 13
40 -- 26
40 -- 27
40 -- 57
40 -- 74
54 -- 1
54 -- 11
54 -- 13
54 -- 26
54 -- 57
54 -- 74
57 -- 40
57 -- 54
74 -- 40
74 -- 54


In [265]:
ours_df[ours_df['Token Id'] == 54]

,Unnamed: 0,Stamina,Special Set,Pet,Mask,Clothes,Eyes,Mouth,Antenna,Skin,Background,Class,Rarity score,Token Id
54,54,4293.24,Shark-Onesie,None,None,Shark-Onesie,Goof,Happy-Fangs,None,Silver,Electric-Purple,Elite,73.681974,54


In [266]:
ours_df[ours_df['Special Set'] != 'None'].shape

(14, 14)

In [268]:
ours_df.isna().any()

Unnamed: 0      False
Stamina         False
Special Set     False
Pet             False
Mask            False
Clothes         False
Eyes            False
Mouth           False
Antenna         False
Skin            False
Background      False
Class           False
Rarity score    False
Token Id        False
dtype: bool